# Integration of nonlinear elasticity equations

Soil is often modeled as having pressure-dependent shear and bulk moduli. Integrating these equations poses some challenges in return mapping algorithms. This notebook explores a common functional form for pressure dependence of shear and bulk modulus and derives updating equations for the elastic stresses.

## Functional form

$K = K_{ref}\left(\frac{p}{p_a}\right)^m$  

$G = G_{ref}\left(\frac{p}{p_a}\right)^n$  

where  

$K_{ref}$ is the reference bulk modulus, and $G_{ref}$ is the reference shear modulus at $p = p_a$.  

Note that  

$K_{ref} = G_{ref}\frac{1-2\nu}{2\left(1+\nu\right)}$  

## Rate equations

$\dot{p} = K \dot{\epsilon_v^e}$  

$\dot{q} = 3G \dot{\epsilon_q^e}$  

## Integrating rate equation for $\epsilon_v^e$ term

Note that

$\int_{t_i}^{t_{i+1}} \frac{dp}{dt} dt = p_{i+1} - p_{i}$

Rate equation after substituting expression for $K$

$\frac{\dot{p}}{p^m} = \frac{K_{ref}}{p_a^m}\dot{\epsilon_v^e}$  

Integrate both sides  

$\int_{p_i}^{p_{i+1}} p^{-m} dp = \int_{\epsilon_v,i^e}^{\epsilon_{v,i+1}^e} \frac{K_{ref}}{p_a^m} d\epsilon_v^e$ 

$\frac{p_{i+1}^{1-m} - p_{i+1}^{1-m}}{1-m} = \frac{K_{ref}}{p_a^m} d\epsilon_v^e$  

Solve for $p_{i+1}$  

$p_{i+1} = exp\left[\frac{ln\left[p_i^{1-m} + \frac{ \left(1-m\right) K_{ref}}{p_a^m} d\epsilon_v^e\right]}{1-m}\right]$  

note: if m = 1  

$p_{i+1} = p_i exp\left(\frac{K_{ref}}{p_a^m} d\epsilon_v^e\right)$  



## Validation using explicit integration and many time steps

In [65]:
import numpy as np
import matplotlib.pyplot as plt

# validate rate equation for epsilon_v^e term using explicit integration
Gref = 2.0*200**2
nu = 0.3
Kref = Gref*(1-2*nu)/(2*(1+nu))
m = 1.0
pa = 101.325

N = 10000
deps_ve = 0.01
epsilon_ve = np.linspace(0,deps_ve,N)
po = 100.0
p_ex = po
for i in range(1,N):
    depsilon_ve = epsilon_ve[i] - epsilon_ve[i-1]
    K = Kref*(p_ex/pa)**m
    p_ex += K*depsilon_ve
print('Explicit integration using ' + str(N) + ' time steps: ' + str(p_ex))

if(m!=1):
    pi1 = np.exp(np.log(po**(1-m) + (1-m)*Kref/pa**m*deps_ve)/(1-m))
else:
    pi1 = po*np.exp(Kref/pa*deps_ve)

print('Exact solution: ' + str(pi1))

print('Forward Euler method using one time step: ' + str(po + Kref*(po/pa)**m*deps_ve))

Explicit integration using 10000 time steps: 336.894964135377
Exact solution: 336.9198188534109
Forward Euler method using one time step: 221.46747898043233


# Integrating rate equations for $\epsilon_q^e$ term

$\frac{\dot{q}}{\dot{p}} = \frac{3G\dot{\epsilon_q^e}}{K\dot{\epsilon_v^e}}$  

$\frac{G}{K} = \frac{G_{ref}}{K_{ref}}\left(\frac{p}{p_a}\right)^{n-m}$  

$\dot{q} = \frac{3G_{ref}}{K_{ref}}\frac{\dot{\epsilon_q^e}}{\dot{\epsilon_v^e}}\dot{p}\left(\frac{p}{p_a}\right)^{n-m}$  

integrate both sides  

$\int_{q_i}^{q_{i+1}}dq = \frac{3G_{ref}}{K_{ref}}\frac{\dot{\epsilon_q^e}}{\dot{\epsilon_v^e}}\int_{p_i}^{p_{i+1}}\left(\frac{p}{p_a}\right)^{n-m}dp$  

evaluate and solve for $q_{i+1}$  

$q_{i+1} = q_i + \frac{3G_{ref}}{K_{ref}}\frac{\dot{\epsilon_q^e}}{\dot{\epsilon_v^e}}\frac{p_a^{m-n}}{1+n-m}\left(p_{i+1}^{1+n-m} - p_i^{1+n-m}\right)$  

note, if m = n, we have $\nu$ = const, and the equation reduces to:  

$q_{i+1} = q_i + \frac{3G_{ref}}{K_{ref}}\frac{\dot{\epsilon_q^e}}{\dot{\epsilon_v^e}}\left(p_{i+1}-p_{i}\right)$  

furthermore, in the case that $\dot{\epsilon_v^e} = 0$, G is constant during shearing and we have:  

$q_{i+1} = q_i + 3G_{ref}\left(\frac{p_i}{p_a}\right)^n\dot{\epsilon_q^e}$  

## Validation using explicit integration and many time steps

In [74]:
import numpy as np
import matplotlib.pyplot as plt

# validate rate equation for epsilon_v^e term using explicit integration
Gref = 2.0*200**2
nu = 0.3
Kref = Gref*(1-2*nu)/(2*(1+nu))
m = 1.0
n = 0.5
pa = 101.325

N = 10000
deps_ve = 0.01
deps_qe = 0.02
epsilon_ve = np.linspace(0,deps_ve,N)
epsilon_qe = np.linspace(0,deps_qe,N)
po = 100.0
qo = 0.0
p_ex = po
q_ex = qo
for i in range(1,N):
    depsilon_ve = epsilon_ve[i] - epsilon_ve[i-1]
    depsilon_qe = epsilon_qe[i] - epsilon_qe[i-1]
    K = Kref*(p_ex/pa)**m
    G = Gref*(p_ex/pa)**n
    p_ex += K*depsilon_ve
    q_ex += 3*G*depsilon_qe
print('Explicit integration using ' + str(N) + ' time steps: p = ' + str(p_ex))

if(m!=1):
    pi1 = np.exp(np.log(po**(1-m) + (1-m)*Kref/pa**m*deps_ve)/(1-m))
else:
    pi1 = po*np.exp(Kref/pa*deps_ve)
print('Exact solution: p = ' + str(pi1))

if(m==n):
    qi1 = qo + 3*Gref/Kref*deps_qe/deps_ve*(pi1-po)
elif(deps_ve ==0):
    qi1 = qo + 3*Gref*(po/pa)**n*deps_qe
else:
    qi1 = qo + 3*Gref/Kref*deps_qe/deps_ve*pa**(m-n)/(1+n-m)*(pi1**(1+n-m) - po**(1+n-m))
print('Explicit integration using ' + str(N) + ' time steps: q = ' + str(q_ex))
print('Exact solution: q = ' + str(qi1))


Explicit integration using 10000 time steps: p = 336.894964135377
Exact solution: p = 336.9198188534109
Explicit integration using 10000 time steps: q = 6559.895027281174
Exact solution: q = 6560.2274055912985
